In [1]:
import dask
import numpy as np
import pandas as pd

# Define tstore_ids
tstore_ids = np.arange(1, 5)

# Define dataframe with attributes
data = {"var1": ["A", "B", "C", "D"], "var2": [1.0, 2.0, 3.0, 4.0]}
df_attrs = pd.DataFrame(data, index=tstore_ids)
df_attrs

,var1,var2
1,A,1.0
2,B,2.0
3,C,3.0
4,D,4.0


In [2]:
# Create TSArray
ts_dict = {}
for tstore_id in tstore_ids:
    df_dask = dask.datasets.timeseries(
        start="2000-01-01",
        end="2000-01-31",
        freq="1s",
        partition_freq="1d",
        dtypes=None,
        seed=None,
    )
    ts_dict[tstore_id] = tstore.TS(df_dask)
ts_dict

NameError: name 'tstore' is not defined

In [ ]:
# Create TSArray
data = list(ts_dict.values())
ts_arr = tstore.TSArray(data)
ts_arr

In [ ]:
# Create Series with TS Dtype
df_series = pd.Series(ts_arr, index=tstore_ids)
print(df_series)  # dtype: TS[dask.DataFrame]  ...  pandas/polars/pyarrow
print(df_series.dtype)  # TSDtype
print(df_series.values)  # TSArray
print(df_series.array)  # TSArray # zero-copy reference to the data !
df_series.to_numpy()  # object dtype

In [ ]:
# Join to df_attrs
ts_variable = "precipitation"
df_attrs[ts_variable] = df_series

In [ ]:
# Create TSDF
tsdf = tstore.TSDF(df_attrs)
dir(tsdf)

In [ ]:
# Write TStore
base_dir = "/tmp/dummy_tstore"
partition_str = None  # TO IMPLEMENT
tstore_structure = "id-var"
overwrite = True
tsdf.to_tstore(
    base_dir,
    partition_str=partition_str,
    tstore_structure=tstore_structure,
    overwrite=overwrite,
)

In [ ]:
# Open TSDF
tsdf = tstore.open_tsdf(base_dir)

In [ ]:
# Open TSDF long
from tstore.tslong.pandas import open_tslong

open_tslong(base_dir, ts_variables=["precipitation"])

# Netatmo data

## Preprocess data

### Utils

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import glob
import tempfile
from os import path

import pandas as pd
import pyarrow as pa
import tstore
from dask import dataframe as dd
from meteostations_vector_cube import netatmo_utils

### Load multi-variate data from list of JSON responses

In [5]:
data_dir = "../data/raw/netatmo-lausanne-aug-21"

In [6]:
data_filepaths = glob.glob(path.join(data_dir, "*.json"))

ts_df, station_gser = netatmo_utils.process_filepaths(data_filepaths)
# drop stations with the same location (likely due to an incorrect set up that led to
# automatic location assignment based on the IP address of the wireless network)
station_gser = station_gser[~station_gser.duplicated(keep=False)]
ts_df = ts_df.loc[:, station_gser.index, :]

  0%|                                                                                                                                                | 0/310 [00:00<?, ?it/s]/home/martibosch/mambaforge/envs/meteostations-vector-cube/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/home/martibosch/mambaforge/envs/meteostations-vector-cube/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/home/martibosch/mambaforge/envs/meteostations-vector-cube/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single 

/home/martibosch/mambaforge/envs/meteostations-vector-cube/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/home/martibosch/mambaforge/envs/meteostations-vector-cube/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
  9%|████████████▋                                                                                                                          | 29/310 [00:00<00:05, 50.05it/s]/home/martibosch/mambaforge/envs/meteostations-vector-cube/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single 

 21%|████████████████████████████▋                                                                                                          | 66/310 [00:01<00:04, 58.13it/s]/home/martibosch/mambaforge/envs/meteostations-vector-cube/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/home/martibosch/mambaforge/envs/meteostations-vector-cube/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
 24%|████████████████████████████████▏                                                                                                      | 74/310 [00:01<00:03, 64.00it/s]/home/martibosch/mambaforge/envs/meteostations

/home/martibosch/mambaforge/envs/meteostations-vector-cube/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
 46%|██████████████████████████████████████████████████████████████▏                                                                       | 144/310 [00:02<00:03, 54.39it/s]/home/martibosch/mambaforge/envs/meteostations-vector-cube/lib/python3.11/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
 52%|█████████████████████████████████████████████████████████████████████▌                                                                | 161/310 [00:02<00:02, 65.69it/s]/home/martibosch/mambaforge/envs/meteostations

we now have two pandas objects:
- `ts_df` is a dataframe with the time series of station measurements, indexed by variable name, station id and time
- `station_gser` is a series with the station location, indexed by station id

In [7]:
ts_df.head()

value
variable id                time                      
humidity 70:ee:50:00:10:f0 2021-08-12 13:00:06   50.0
                           2021-08-12 14:00:11   49.0
                           2021-08-12 15:00:16   50.0
                           2021-08-12 16:00:12   49.0
                           2021-08-12 17:00:10   53.0

now, for each station, convert its respective the dataframe to dask and instantiate a tstore object with it; put the tstore objects in a dictionary indexed by station id

In [8]:
id_col = "id"
time_col = "time"
chunksize = 100
ts_dict = {
    _id: tstore.TS(
        dd.from_pandas(df.drop(id_col, axis=1), chunksize=chunksize).set_index(time_col)
    )
    for _id, df in ts_df.reset_index().groupby(id_col)
}

TSArray

In [ ]:
df_ser = pd.Series(tstore.TSArray(list(ts_dict.values())), index=ts_dict.keys())
df_ser

In [ ]:
var_col = "variable"
df_attrs = pd.DataFrame()
for var, var_df in ts_df.reset_index().groupby(var_col):
    id_gb = var_df.groupby(id_col)
    df_attrs[var] = pd.Series(
        tstore.TSArray(
            [
                tstore.TS(
                    dd.from_pandas(
                        df.drop([id_col, var_col], axis=1), chunksize=chunksize
                    ).set_index(time_col)
                )
                for _, df in id_gb
            ]
        ),
        index=[_id for _id, _ in id_gb],
    )
tsdf = tstore.TSDF(df_attrs)

In [ ]:
base_dir = "/tmp/dummy_tstore"
partition_str = None  # TO IMPLEMENT
tstore_structure = "id-var"
overwrite = True
tsdf.to_tstore(
    base_dir,
    partition_str=partition_str,
    tstore_structure=tstore_structure,
    overwrite=overwrite,
)

In [ ]:
tsdf = tstore.open_tsdf(base_dir)

In [ ]:
tsdf["temperature"].iloc[0]

In [ ]:
tslong = open_tslong(base_dir, ts_variables=["humidity"])
tslong


## 1. user-friendly interface to instantiating:
   - TSDF: from a dataframe
   - GeoTSDF: from a dataframe and a geoseries

Starting point: we have ts_df and station_gser
### Option a)
- gpd.GeoDataFrame.to_parquet

In [ ]:
# gpd.GeoDataFrame(ts_df
for group, group_df in ts_df.reset_index().groupby("id"):
    break

In [ ]:
for ggroup, ggroup_df in group_df.drop("id", axis=1).groupby("variable"):
    break
ggroup_df.drop("variable", axis=1).set_index("time").squeeze()  # .rename(ggroup)

In [ ]:
import geopandas as gpd

In [ ]:
gpd.GeoDataFrame(
    ts_df.reset_index()
    .groupby("id")
    .apply(
        lambda group_df: group_df.drop("id", axis=1)
        .groupby("variable")
        .apply(lambda ggroup_df: ggroup_df.drop("variable", axis=1).set_index("time"))
    )
    .reset_index(level=("variable", "time")),
    geometry=station_gser,
)

In [ ]:
ts_df.reset_index().groupby("id").apply(
    lambda station_df: station_df.drop("id", axis=1)
    .groupby("variable")
    .apply(lambda var_df: var_df.drop("variable", axis=1).set_index("time"))
)

In [ ]:
# gpd.GeoDataFrame(foo, geometry=station_gser)
{
    variable: ts_ser
    for variable, ts_ser in group_df.drop("id", axis=1)
    .groupby("variable")
    .apply(lambda ggroup_df: ggroup_df.drop("variable", axis=1).set_index("time"))
}

In [ ]:
group_df.drop("id", axis=1).groupby("variable").apply(
    lambda ggroup_df: ggroup_df.drop("variable", axis=1).set_index("time")
)

In [ ]:
# pd.DataFrame({_id: ts_dict.items()))
pd.DataFrame(
    {
        _id: tstore.TS(dd.from_pandas(df.drop(id_col, axis=1), chunksize=chunksize))
        for _id, df in ts_df.reset_index().groupby(id_col)
    }
)

In [ ]:
tsdf = tstore.TSDF(pd.DataFrame(ts_dict), id_col=id_col)

In [ ]:
# transform ts_df to dask partitioning by station id
#
dd.from_pandas(
    ts_df.reset_index().set_index("id"),
)

In [ ]:
def _add_partition_cols(ts_df):
    if "time" not in ts_df.columns:
        ts_df = ts_df.reset_index(level="time")
    else:
        ts_df = ts_df.copy()  # avoid modifying the original dataframe
    partition_cols = ["year", "month"]
    for partition_col in partition_cols:
        ts_df[partition_col] = getattr(ts_df["time"].dt, partition_col)
    return ts_df, partition_cols


def write_parquet(ts_df, filepath):
    ts_df, partition_cols = _add_partition_cols(ts_df)

    ts_df.to_parquet(
        filepath,
        engine="pyarrow",
        write_index=True,
        custom_metadata=None,
        write_metadata_file=True,
        name_function=lambda i: f"part.{i}.parquet",
        partition_on=partition_cols,
        # Encoding
        schema="infer",
        compression="snappy",
        # Writing options
        append=False,
        overwrite=False,
        ignore_divisions=False,
        compute=True,
    )

In [ ]:
df, cols = _add_partition_cols(ts_df)

In [ ]:

with tempfile.TemporaryDirectory() as tmp_dir:
    filepath = path.join(tmp_dir, "netatmo.parquet")
    write_parquet(ts_df, filepath)